In [ ]:
# Install dependencies from pyproject.toml (works on Google Colab)
%pip install -q -e .

In [ ]:

from pathlib import Path
import imageio
import gymnasium as gym
import minigrid
from minigrid.wrappers import ImgObsWrapper
from minigrid.wrappers import RGBImgPartialObsWrapper
import random
import numpy as np
import torch as th


import torch.nn as nn
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
from stable_baselines3.common.vec_env import DummyVecEnv, VecMonitor, VecNormalize

# Set random seed for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
th.manual_seed(SEED)
th.cuda.manual_seed_all(SEED)
th.backends.cudnn.deterministic = True
th.backends.cudnn.benchmark = False

CURRENT_DIRECTORY = Path(".")
model_output = CURRENT_DIRECTORY / "ppo_minigrid.zip"
LOG_DIR = CURRENT_DIRECTORY / "logs"
LOG_DIR.mkdir(exist_ok=True)

# uncomment following line to retrain the model
# model_output.unlink(missing_ok=True)


def make_env(seed: int = 0, log_suffix: str = ""):
	"""Create a monitored MiniGrid environment with RGB partial observations."""
	env = gym.make("MiniGrid-Empty-8x8-v0", render_mode="rgb_array", max_episode_steps=100)
	env = RGBImgPartialObsWrapper(env)
	env = ImgObsWrapper(env)
	log_path = LOG_DIR / f"train{log_suffix}"
	log_path.parent.mkdir(parents=True, exist_ok=True)
	env = Monitor(env, log_path.as_posix())
	env.reset(seed=seed)
	env.action_space.seed(seed)
	env.observation_space.seed(seed)
	return env


# Official recommendation: custom feature extractor for MiniGrid image input
class MinigridFeaturesExtractor(BaseFeaturesExtractor):
	def __init__(self, observation_space, features_dim: int = 256):
		super().__init__(observation_space, features_dim)
		n_input_channels = observation_space.shape[0]
		self.cnn = nn.Sequential(
			nn.Conv2d(n_input_channels, 32, kernel_size=3, stride=1, padding=1),
			nn.ReLU(),
			nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
			nn.ReLU(),
			nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
			nn.ReLU(),
			nn.Flatten(),
		)
		with th.no_grad():
			sample = th.as_tensor(observation_space.sample()[None]).float()
			n_flatten = self.cnn(sample).shape[1]
		self.linear = nn.Sequential(nn.Linear(n_flatten, features_dim), nn.ReLU())

	def forward(self, observations):
		return self.linear(self.cnn(observations))


policy_kwargs = dict(
	features_extractor_class=MinigridFeaturesExtractor,
	features_extractor_kwargs=dict(features_dim=256),
	normalize_images=True,
)


# Create MiniGrid-Empty-8x8-v0 environment with image observation
vec_env = DummyVecEnv([lambda seed=SEED + i: make_env(seed, f"_{i}") for i in range(8)])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=True, clip_obs=10.0)
train_env = VecMonitor(vec_env, LOG_DIR.as_posix())


if not model_output.exists():
	print("Training new model...")
	model = PPO(
		"CnnPolicy",
		train_env,
		policy_kwargs=policy_kwargs,
		learning_rate=2.5e-4,
		n_steps=1024,
		batch_size=1024,
		gamma=0.995,
		gae_lambda=0.9,
		clip_range=0.2,
		ent_coef=0.001,
		vf_coef=0.5,
		max_grad_norm=0.4,
		verbose=1,
	)
	model.learn(total_timesteps=int(1e6), progress_bar=True)
	model.save(model_output)
else:
	print("Model already exists, loading...")
	model = PPO.load(model_output, env=train_env)


# Evaluate the agent
eval_env = DummyVecEnv([lambda seed=1000 + i: make_env(seed, f"_eval_{i}") for i in range(4)])
eval_env = VecNormalize(eval_env, training=False, norm_obs=True, norm_reward=True, clip_obs=10.0)
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100, render=False)
print(f"Mean reward: {mean_reward} +/- {std_reward}")



Training new model...
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Output()

/usr/local/lib/python3.12/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 252      |
|    ep_rew_mean     | 0.0266   |
| time/              |          |
|    fps             | 369      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------


...

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 11.5        |
|    ep_rew_mean          | 0.96        |
| time/                   |             |
|    fps                  | 266         |
|    iterations           | 95          |
|    time_elapsed         | 729         |
|    total_timesteps      | 194560      |
| train/                  |             |
|    approx_kl            | 0.032975264 |
|    clip_fraction        | 0.0102      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0127     |
|    explained_variance   | 0.958       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0219     |
|    n_updates            | 940         |
|    policy_gradient_loss | -0.0126     |
|    value_loss           | 2.46e-05    |
-----------------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 12.8        |
|    ep_rew_mean          | 0.955       |
| time/                   |             |
|    fps                  | 266         |
|    iterations           | 96          |
|    time_elapsed         | 737         |
|    total_timesteps      | 196608      |
| train/                  |             |
|    approx_kl            | 0.012480363 |
|    clip_fraction        | 0.0397      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.0947     |
|    explained_variance   | 0.871       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0257     |
|    n_updates            | 950         |
|    policy_gradient_loss | -0.0117     |
|    value_loss           | 0.00012     |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 11.2       |
|    ep_rew_mean          | 0.961      |
| time/                   |            |
|    fps                  | 266        |
|    iterations           | 97         |
|    time_elapsed         | 745        |
|    total_timesteps      | 198656     |
| train/                  |            |
|    approx_kl            | 0.29927137 |
|    clip_fraction        | 0.303      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.0825    |
|    explained_variance   | 0.744      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0462    |
|    n_updates            | 960        |
|    policy_gradient_loss | -0.0461    |
|    value_loss           | 0.000271   |
----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 13.3       |
|    ep_rew_mean          | 0.953      |
| time/                   |            |
|    fps                  | 266        |
|    iterations           | 98         |
|    time_elapsed         | 753        |
|    total_timesteps      | 200704     |
| train/                  |            |
|    approx_kl            | 0.11365476 |
|    clip_fraction        | 0.0964     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.065     |
|    explained_variance   | 0.964      |
|    learning_rate        | 0.0003     |
|    loss                 | -0.0354    |
|    n_updates            | 970        |
|    policy_gradient_loss | -0.0221    |
|    value_loss           | 6.11e-05   |
----------------------------------------


Mean reward: 0.0 +/- 0.0


In [11]:
def record_video(env, agent):
	frames = []
	state, _ = env.reset()
	# Record the first frame
	frames.append(env.render())

	done = False
	while not done:
		action, _ = agent.predict(state, deterministic=True)
		state, _, terminated, truncated, _ = env.step(action)
		frames.append(env.render())
		done = terminated or truncated

	# Save the recorded frames as a video
	imageio.mimsave(CURRENT_DIRECTORY / "demo.mp4", frames, fps=1)

record_video(env, model)

In [12]:
from IPython.display import Video

Video("./demo.mp4", embed=True)